# Temporal error evolution

> Authors: Alexander Geiss
>
> Abstract: Using AUX_MET data to determine the temporal evolution of L2B systematic and random errors for longer time periods.  

This notebook is currently "in progress" and covers the loading of the data for now.

## Load packages, modules and extensions

In [ ]:
# %load_ext blackcellmagic
# enable following line for interactive visualization backend for matplotlib
# %matplotlib widget
# print version info
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib

In [ ]:
from viresclient import AeolusRequest
import numpy as np
from scipy.stats import linregress, median_abs_deviation
import netCDF4 as nc
import xarray as xr
import pandas as pd
from datetime import datetime, timezone
import os
import dask
import matplotlib.pyplot as plt
from base64 import b64decode
from matplotlib.collections import PolyCollection
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.colors as colors
import cartopy.crs as ccrs
from ipywidgets import interact
import ipywidgets as widgets
from tqdm.notebook import tqdm_notebook

# Create time-series of random and systematic error by using AUX-MET data

This task can be divided into several steps:

1. Loading data
2. Collocating data
    - Temporal collocation (same orbits)
    - Spatial collocation (horizontal/vertical)
3. Filtering, Analyzing and Plotting data


## Load AUX_MET and Aeolus L2B data
For a detailed description on how to load AUX_MET and L2B data, please see the corresponding notebooks.  
Computing long-term statistics of Aeolus' errors requires a different strategy for loading and handling large data sets with the available system resources. One option is to create requests on a monthly basis and save the retrieved data directly as netcdf data in the user space. Because some of the data within one month can be incomplete this would lead to a failed request for the whole month. Therefore, we reduce each request to half-month periods.
The retrieved and saved data can then be loaded with xarray and dask functionality to concatenate and process the files. Here it must be taken care of saved but invalid data which could be empty.

In [ ]:
# set dask scheduler to single-threaded since we only have one core available
dask.config.set(scheduler='single-threaded')

In [ ]:
# create start and end dates (here for 3 months of data)
dates = pd.date_range(start='2021-12-01', end='2022-03-01', freq='SMS').to_pydatetime()
start_dates = dates[:-1]
end_dates = dates[1:]

### Load L2B data
Define fields

In [ ]:
# Aeolus product
DATA_PRODUCT = "ALD_U_N_2B"

# define fields to retrieve
parameter_rayleigh = [
    "rayleigh_wind_result_start_time",
    "rayleigh_wind_result_stop_time",
    "rayleigh_wind_result_top_altitude",
    "rayleigh_wind_result_bottom_altitude",
    "rayleigh_wind_result_start_latitude",
    "rayleigh_wind_result_start_longitude",
    "rayleigh_wind_result_stop_latitude",
    "rayleigh_wind_result_stop_longitude",
    "rayleigh_wind_result_range_bin_number",
    "rayleigh_wind_result_wind_velocity",
    "rayleigh_wind_result_HLOS_error",
    "rayleigh_wind_result_los_azimuth",
    "rayleigh_wind_result_observation_type",
    "rayleigh_wind_result_validity_flag",
]

parameter_mie = [
    "mie_wind_result_start_time",
    "mie_wind_result_stop_time",
    "mie_wind_result_top_altitude",
    "mie_wind_result_bottom_altitude",
    "mie_wind_result_start_latitude",
    "mie_wind_result_start_longitude",
    "mie_wind_result_stop_latitude",
    "mie_wind_result_stop_longitude",
    "mie_wind_result_range_bin_number",
    "mie_wind_result_wind_velocity",
    "mie_wind_result_HLOS_error",
    "mie_wind_result_los_azimuth",
    "mie_wind_result_observation_type",
    "mie_wind_result_validity_flag",
]

Generate file names and paths for saving data periods as netCDF files.  

In [ ]:
L2B_rayleigh_filenames = [
    f"L2B_rayleigh_{start.strftime('%Y-%m-%d')}-{end.strftime('%Y-%m-%d')}.nc"
    for start, end in zip(start_dates, end_dates)
]
L2B_mie_filenames = [
    f"L2B_mie_{start.strftime('%Y-%m-%d')}-{end.strftime('%Y-%m-%d')}.nc"
    for start, end in zip(start_dates, end_dates)
]

user_folder = os.path.expanduser("~")
L2B_folder = os.path.join(user_folder, "files/L2B")
os.makedirs(L2B_folder, exist_ok=True)
L2B_rayleigh_file_paths = [
    os.path.join(L2B_folder, filename) for filename in L2B_rayleigh_filenames
]
L2B_mie_file_paths = [os.path.join(L2B_folder, filename) for filename in L2B_mie_filenames]

**Request L2B Rayleigh data**  
This can take some time depending on the requested time period and region. 
Tested for the whole mission time from 01.09.2018 until 01.02.2022 for the region of Central Europe which took around 30 minutes.   
Here we repeat the get_between method for our list of half-months and save the retrieved (netCDF)-files to our user space right away using the to_file method.
The data for the requested period is only retrieved and saved if it is not yet available under the defined file path.
The list where the file paths are saved is then checked for files which are empty (have no groups in case of L2B data).

In [ ]:
# Create Aeolus request
request = AeolusRequest()
request.set_collection(DATA_PRODUCT)
request.set_fields(rayleigh_wind_fields=parameter_rayleigh)

# Set bounding box for region around Central Europe
request.set_bbox({"n": 60, "w": 0, "s": 45, "e": 20})

# create list which will hold the loaded Rayleigh data filenames
L2B_rayleigh_file_paths_loaded = []

# loop over all half-months and request the data
for measurement_start, measurement_stop, filename in zip(
    start_dates, end_dates, L2B_rayleigh_file_paths
):
    # check if file already exists
    # if not, then request the data
    if not os.path.isfile(filename):
        try:
            L2B_rayleigh_data = request.get_between(
                start_time=measurement_start,
                end_time=measurement_stop,
                filetype="nc",
                asynchronous=True,
            ).to_file(filename, overwrite=True)
            L2B_rayleigh_file_paths_loaded.append(filename)
        except Exception as e:
            print(f"No data for the period {measurement_start}-{measurement_stop} available")
            # context is printed here because it holds the true reason for the Exception
            print(e.__context__)
    # if file already exists, append the filename to the list
    else:
        L2B_rayleigh_file_paths_loaded.append(filename)

# Check file list for empty files (no groups available) and remove them from the list
L2B_rayleigh_file_paths_valid = [i for i in L2B_rayleigh_file_paths_loaded if nc.Dataset(i).groups]

**Request L2B Mie data**  
Same procedure as above but now for Mie wnd data.

In [ ]:
# Create Aeolus request
request = AeolusRequest()
request.set_collection(DATA_PRODUCT)
request.set_fields(mie_wind_fields=parameter_mie)

# Set bounding box for region around Germany
request.set_bbox({"n": 60, "w": 0, "s": 45, "e": 20})

# create list which will hold the loaded Mie data
L2B_mie_file_paths_loaded = []
for measurement_start, measurement_stop, filename in zip(
    start_dates, end_dates, L2B_mie_file_paths
):
    # check if file already exists
    if not os.path.isfile(filename):
        try:
            L2B_mie_data = request.get_between(
                start_time=measurement_start,
                end_time=measurement_stop,
                filetype="nc",
                asynchronous=True,
            ).to_file(filename, overwrite=True)
            L2B_mie_file_paths_loaded.append(filename)
        except Exception as e:
            print(f"No data for the period {measurement_start}-{measurement_stop} available")
            print(e.__context__)
    else:
        L2B_mie_file_paths_loaded.append(filename)

# Check file list for empty files
L2B_mie_file_paths_valid = [i for i in L2B_mie_file_paths_loaded if nc.Dataset(i).groups]

**Read netCDF-data as xarray dataset (dask array)**  
xarray.open_mfdataset can be used to read multiple netCDF files into a single xarray dataset. Because data is stored in groups, they must be explicitly passed as parameter to the open_mfdataset method. 
The group names can be checked with the netCDF4 module and are depending on the requested fields.
The dimension which shall be used for concatenation, must also be defined. Data is then loaded as Dask arrays. (https://xarray.pydata.org/en/stable/user-guide/dask.html)

In [ ]:
%%time
ds_rayleigh = xr.open_mfdataset(
    L2B_rayleigh_file_paths_valid,
    group="rayleigh_wind_data",
    combine="nested",
    concat_dim="rayleigh_wind_data",
)
ds_mie = xr.open_mfdataset(
    L2B_mie_file_paths_valid, group="mie_wind_data", combine="nested", concat_dim="mie_wind_data",
)

&#9889; **Please note that HLOS error is saved in m/s instead of cm/s for dates before 15.06.2019**  
With the reprocessing of these time periods, this problem should be solved.  
But for now we have to convert HLOS errors to cm/s for these dates which is necessary for further processing and applying QC thresholds.

In [ ]:
%%time
rayleigh_hlos_error = ds_rayleigh['rayleigh_wind_result_HLOS_error'].values
rayleigh_hlos_error[(ds_rayleigh['rayleigh_wind_result_start_time'] < 6.14e8).values] *= 100
ds_rayleigh['rayleigh_wind_result_HLOS_error'] = (('rayleigh_wind_data'), rayleigh_hlos_error)

mie_hlos_error = ds_mie['mie_wind_result_HLOS_error'].values
mie_hlos_error[(ds_mie['mie_wind_result_start_time'] < 6.14e8).values] *= 100
ds_mie['mie_wind_result_HLOS_error'] = (('mie_wind_data'), mie_hlos_error)

&#9889; **Please note that there was a height offset present for dates before 26.02.2019**  
Check following document for more information on needed corrections: [The NWP impact of Aeolus Level-2B Winds at ECMWF - Technical memorandum](https://www.ecmwf.int/file/288329/download?token=y9cKewWP) by M. Rennie and L. Isaksen (here section 2.4.1.2)  
With the reprocessing of these time periods, this problem should be solved.

In [ ]:
rayleigh_top_altitude = ds_rayleigh["rayleigh_wind_result_top_altitude"].values
rayleigh_bottom_altitude = ds_rayleigh["rayleigh_wind_result_bottom_altitude"].values
rayleigh_top_altitude[(ds_rayleigh["rayleigh_wind_result_top_altitude"] < 6.04e8).values] += 250
rayleigh_bottom_altitude[
    (ds_rayleigh["rayleigh_wind_result_bottom_altitude"] < 6.04e8).values
] += 250
ds_rayleigh["rayleigh_wind_result_top_altitude"] = (("rayleigh_wind_data"), rayleigh_top_altitude)
ds_rayleigh["rayleigh_wind_result_bottom_altitude"] = (
    ("rayleigh_wind_data"),
    rayleigh_bottom_altitude,
)

mie_top_altitude = ds_mie["mie_wind_result_top_altitude"].values
mie_bottom_altitude = ds_mie["mie_wind_result_bottom_altitude"].values
mie_top_altitude[(ds_mie["mie_wind_result_top_altitude"] < 6.04e8).values] += 250
mie_bottom_altitude[(ds_mie["mie_wind_result_bottom_altitude"] < 6.04e8).values] += 250
ds_mie["mie_wind_result_top_altitude"] = (("mie_wind_data"), mie_top_altitude)
ds_mie["mie_wind_result_bottom_altitude"] = (("mie_wind_data"), mie_bottom_altitude)

### Load AUX_MET data
Define fields

In [ ]:
# Aeolus product
DATA_PRODUCT = "AUX_MET_12"

# off-nadir parameter
parameter_off_nadir = [
    "time",
    "latitude",
    "longitude",
    "surface_altitude",
    "layer_altitude",
    "layer_wind_component_u",
    "layer_wind_component_v",
]
parameter_off_nadir = [param + "_off_nadir" for param in parameter_off_nadir]

Generate file names and paths for saving data periods as netCDF files.  

In [ ]:
AUX_MET_filenames = [
    f"AUX_MET_{start.strftime('%Y-%m-%d')}-{end.strftime('%Y-%m-%d')}.nc"
    for start, end in zip(start_dates, end_dates)
]
user_folder = os.path.expanduser("~")
AUX_MET_folder = os.path.join(user_folder, "files/AUX_MET")
os.makedirs(AUX_MET_folder, exist_ok=True)
AUX_MET_file_paths = [os.path.join(AUX_MET_folder, filename) for filename in AUX_MET_filenames]

**Request AUX_MET data**  
This can take some time depending on the requested time period and region. For the whole mission it takes around 30 minutes.
Here we choose a slightly larger bounding box (1 degree) to be sure to have AUX_MET data within all L2B observations which is especially important at the edges of our L2B bounding box.

In [ ]:
# Create Aeolus request
request = AeolusRequest()
request.set_collection(DATA_PRODUCT)
request.set_fields(fields=parameter_off_nadir)

# Set bounding box for region around Germany
request.set_bbox({"n": 61, "w": -1, "s": 44, "e": 21})

# create list which will hold the loaded AUX_MET data
AUX_MET_file_paths_loaded = []
for measurement_start, measurement_stop, filename in zip(
    start_dates, end_dates, AUX_MET_file_paths
):
    # check if file already exists
    if not os.path.isfile(filename):
        try:
            AUX_MET_data = request.get_between(
                start_time=measurement_start,
                end_time=measurement_stop,
                filetype="nc",
                asynchronous=True,
            ).to_file(filename, overwrite=True)
            AUX_MET_file_paths_loaded.append(filename)
        except RuntimeError:
            print(f"No data for the period {measurement_start}-{measurement_stop} available")
    else:
        AUX_MET_file_paths_loaded.append(filename)

# Check file list for empty files (here if variables exist)
AUX_MET_file_paths_valid = [i for i in AUX_MET_file_paths_loaded if nc.Dataset(i).variables]

**Read netCDF-data as xarray dataset (dask array)**  
AUX_MET data is not saved in netCDF-groups and thus no group must be passed to the open_mfdataset funtion.

In [ ]:
ds_aux_met = xr.open_mfdataset(AUX_MET_file_paths, combine="nested", concat_dim="off_nadir")

Add datetime and altitude bottom and tops to the AUX_MET dataset for the plotting routines.

In [ ]:
# Add datetime
ds_aux_met["datetime_off_nadir"] = (
    ("off_nadir"),
    nc.num2date(
        ds_aux_met["time_off_nadir"],
        units="s since 2000-01-01",
        only_use_cftime_datetimes=False,
    ),
)

# calculate layer top altitudes
layer_altitude_top = (
    ds_aux_met["layer_altitude_off_nadir"][:, :-1].data
    - ds_aux_met["layer_altitude_off_nadir"][:, :].diff(dim="array_137").data / 2.0
)

# calculate layer bottom altitudes
layer_altitude_bottom = (
    ds_aux_met["layer_altitude_off_nadir"][:, 1:].data
    + ds_aux_met["layer_altitude_off_nadir"][:, :].diff(dim="array_137").data / 2.0
)

# combine bottom and top altitude to get layer borders
layer_altitude_borders = np.concatenate((layer_altitude_top, layer_altitude_bottom[:, -2:]), axis=1)
ds_aux_met["layer_altitude_borders_off_nadir"] = (
    ("off_nadir", "array_138"),
    layer_altitude_borders,
)

## Collocation of AUX_MET and L2B data

AUX_MET data is available with 3 seconds temporal resolution and 137 altitude levels with around 100 levels up to 25 km. That is much higher than the Aeolus vertical resolution of 24 atmospheric range bins.  
In the next steps a temporal and spatially horizontal collocation is needed to look for all AUX_MET profiles within one L2B group.
Subsequently, a vertical averaging to the Aeolus range bin size is needed.  
The L2B line of sight azimuth angle must then be used to calculate the HLOS wind speeds from the wind components in the AUX_MET data.

### Temporal collocation or mapping of same orbits

For collocation and low system memory purposes we want to do computations orbit-wise.  
This makes a separation of the datasets into single orbits necessary.
Instead of creating new datasets for every orbit we just determine the indices for each dataset where new orbits start.
Here we use a time difference of 3600 seconds for consecutive profiles (AUX_MET) or L2B-groups above which we assume orbits are separated. Note that one full orbit takes around 90 minutes and we are looking in this example only at a smaller region.

In [ ]:
# create mask for each data set resulting in an array with a unique index for each orbit.
# The 3600 seconds are hard-coded here.
orbit_mask_l2b_rayleigh = (
    ds_rayleigh["rayleigh_wind_result_start_time"].diff(dim="rayleigh_wind_data") > 3600
).cumsum()
orbit_mask_l2b_mie = (
    ds_mie["mie_wind_result_start_time"].diff(dim="mie_wind_data") > 3600
).cumsum()
orbit_mask_aux_met = (ds_aux_met["time_off_nadir"].diff(dim="off_nadir") > 3600).cumsum()

Because we used the diff-method the resulting masks are smaller in length by one. This must be taken into account by inserting 0 before the first element of the array.

In [ ]:
ds_rayleigh["orbit_mask"] = (
    ds_rayleigh["rayleigh_wind_result_start_time"].dims,
    np.concatenate(([0], orbit_mask_l2b_rayleigh)),
)
ds_mie["orbit_mask"] = (
    ds_mie["mie_wind_result_start_time"].dims,
    np.concatenate(([0], orbit_mask_l2b_mie)),
)
ds_aux_met["orbit_mask"] = (
    ds_aux_met["time_off_nadir"].dims,
    np.concatenate(([0], orbit_mask_aux_met)),
)

To get the indices of the single orbits we determine unique values of our orbit_mask.  
The counts of these unique values can be used to rechunk the Dask arrays to account for single orbits which are then processed orbit-wise to get the collocations.

In [ ]:
# Put the data sets into a dictionary in oder to loop over them
ds_dict = {'rayleigh': ds_rayleigh, 'mie': ds_mie, 'aux_met': ds_aux_met}

In [ ]:
for ds in ds_dict:
    values, counts = np.unique(ds_dict[ds]["orbit_mask"], return_index=False, return_counts=True)
    dim = list(ds_dict[ds].dims)[0]
    ds_dict[ds] = ds_dict[ds].chunk({dim: tuple(counts)})
    if ds == "aux_met":
        ds_dict[ds]["orbit_start_index"] = (("orbit_no"), np.r_[[0], np.cumsum(counts)[:-1]])
        ds_dict[ds]["orbit_stop_index"] = (("orbit_no"), np.cumsum(counts))

In the next step we need to find the collocated orbits for Mie, Rayleigh and AUX_MET data.
AUX_MET data is available for the whole mission without any gaps which is not the case for L2B-data. 
Thus, we would like to map the corresponding orbits in an index-array.

First we have to calculate max, min and median values of the L2B-Rayleigh, -Mie and AUX_MET orbits (divided already in chunks).  
This can be achieved very fast with the map_blocks-Function for dask-arrays.

We need to define a function which calculates the min, max and median values, which can be passed to the map_blocks-function:

In [ ]:
def calc_min_max_median(block):
    return np.array([[np.amin(block), np.amax(block), np.median(block)]])

Apply this function to Rayleigh, Mie and AUX_MET orbits.
This yields an array with length corresponding to the number of orbits (chunks) and 3 values each for min, max and median.

In [ ]:
for ds, time_param in zip(
    ["rayleigh", "mie", "aux_met"],
    ["rayleigh_wind_result_start_time", "mie_wind_result_start_time", "time_off_nadir"],
):
    min_max_median = (
        ds_dict[ds][time_param].data.map_blocks(calc_min_max_median, chunks=(1,)).compute()
    )
    ds_dict[ds]["orbit_times_min"] = (("orbit_no"), min_max_median[:, 0])
    ds_dict[ds]["orbit_times_max"] = (("orbit_no"), min_max_median[:, 1])
    ds_dict[ds]["orbit_times_median"] = (("orbit_no"), min_max_median[:, 2])

Determine the corresponding AUX_MET orbits for L2B Rayleigh and Mie orbits by calculating the time difference between these orbits and choosing the minimum value.
For calculating the difference we use the median. This was sufficient.

In [ ]:
# map the AUX_MET orbits (by index) to the L2B orbits for Rayleigh and Mie
rayleigh_aux_idx = np.argmin(
    np.abs(
        ds_dict["aux_met"]["orbit_times_median"].values
        - ds_dict["rayleigh"]["orbit_times_median"].values[:, np.newaxis]
    ),
    axis=1,
)
mie_aux_idx = np.argmin(
    np.abs(
        ds_dict["aux_met"]["orbit_times_median"].values
        - ds_dict["mie"]["orbit_times_median"].values[:, np.newaxis]
    ),
    axis=1,
)

ds_dict["rayleigh"]["orbit_aux_met_idx"] = (("orbit_no"), rayleigh_aux_idx)
ds_dict["mie"]["orbit_aux_met_idx"] = (("orbit_no"), mie_aux_idx)

We can now check if the time differences between these collocated orbits is smaller than the overpass time for our chosen geolocation box to see if we have found the correct orbits or if there are any outliers.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
ax1.plot(
    nc.num2date(
        ds_dict["aux_met"]["orbit_times_median"][rayleigh_aux_idx],
        units="s since 2000-01-01",
        only_use_cftime_datetimes=False,
    ),
    "o",
    alpha=0.5,
)
ax1.plot(
    nc.num2date(
        ds_dict["rayleigh"]["orbit_times_median"],
        units="s since 2000-01-01",
        only_use_cftime_datetimes=False,
    ),
    ".",
    alpha=0.5,
)
ax1.set_ylabel("date time")
ax1.set_xlabel("orbit no")

ax2.plot(
    nc.num2date(
        ds_dict["rayleigh"]["orbit_times_median"],
        units="s since 2000-01-01",
        only_use_cftime_datetimes=False,
    ),
    ds_dict["aux_met"]["orbit_times_median"][rayleigh_aux_idx]
    - ds_dict["rayleigh"]["orbit_times_median"],
    "o",
    alpha=0.5,
)
ax2.set_xlabel("orbit_no")
ax2.set_ylabel("time difference between orbits [seconds]")
fig.suptitle("Collocated Rayleigh and AUX_MET orbits")

Because all time differences (second subplot) are smaller than 3600 seconds, we have all L2B data with collocated AUX_MET data per orbit or precisely the necessary indices to obtain the AUX_MET data corresponding to the L2B data.

### Horizontal and vertical collocation of L2B wind measurements an AUX_MET data

To see the differences in spatial resolution of L2B and AUX_MET data we have a look at curtain plots of one orbit.
We define a plotting function where the orbit number and several other parameters can be passed.

In [ ]:
def plot_wind_from_collocated_orbits(
    L2B_parameter="rayleigh_wind_result_wind_velocity",
    L2B_channel="rayleigh",
    L2B_obs_type="clear",
    L2B_QC_filter=True,
    L2B_error_estimate_threshold=800,
    orbit_no=0,
    vmin=None,
    vmax=None,
    max_altitude=30,
):
    # define L2B dataset where only the orbit with orbit_no is filtered
    ds_L2B = ds_dict[L2B_channel].where(ds_dict[L2B_channel]["orbit_mask"] == orbit_no, drop=True)

    # get orbit index for AUX_MET data and select the subset
    aux_met_orbit_no = ds_dict[L2B_channel]["orbit_aux_met_idx"][orbit_no]
    ds_aux_met = ds_dict["aux_met"].where(
        ds_dict["aux_met"]["orbit_mask"] == aux_met_orbit_no, drop=True
    )

    # ------- preprocess L2B data -------------------------
    # define necessary L2B parameters for plotting
    X0 = mdates.date2num(
        nc.num2date(
            ds_L2B[L2B_channel + "_wind_result_start_time"].values,
            units="s since 2000-01-01",
            only_use_cftime_datetimes=False,
        )
    )
    X1 = mdates.date2num(
        nc.num2date(
            ds_L2B[L2B_channel + "_wind_result_stop_time"].values,
            units="s since 2000-01-01",
            only_use_cftime_datetimes=False,
        )
    )

    Y0 = ds_L2B[L2B_channel + "_wind_result_bottom_altitude"].values / 1000.0
    Y1 = ds_L2B[L2B_channel + "_wind_result_top_altitude"].values / 1000.0
    Z = ds_L2B[L2B_parameter].values

    # create a mask out of different filters which can be applied to the different parameters
    mask = np.zeros(len(Z), dtype=bool)

    # mask where validity flag is 0
    if L2B_QC_filter:
        mask = mask | (ds_L2B[L2B_channel + "_wind_result_validity_flag"] == 0)

    # mask dependent on observation type
    if L2B_obs_type == "cloudy":
        mask = mask | (ds_L2B[L2B_channel + "_wind_result_observation_type"] != 1)
    elif L2B_obs_type == "clear":
        mask = mask | (ds_L2B[L2B_channel + "_wind_result_observation_type"] != 2)

    # mask where wind results have error estimates larger than a given threshold
    mask = mask | (ds_L2B[L2B_channel + "_wind_result_HLOS_error"] > L2B_error_estimate_threshold)

    # mask all necessary parameters for plotting
    # tilde before mask inverts the boolean mask array
    X0 = X0[~mask]
    X1 = X1[~mask]
    Y0 = Y0[~mask]
    Y1 = Y1[~mask]
    Z = Z[~mask]

    # -------------- preprocess AUX_MET data --------------------------------------
    x_am = ds_aux_met["datetime_off_nadir"]
    altitude_array = ds_aux_met["layer_altitude_borders_off_nadir"] / 100000.0
    ymax = np.mean(np.argmin(np.abs(altitude_array.values - max_altitude), axis=1)).astype(int)
    y_am = altitude_array[:, ymax - 1:]

    # To make AUX_MET model winds comparable to L2B winds we have to account for the horizontal line of sight (HLOS).
    # The model winds u and v component can be converted with the LOS azimuth angle from the L2B wind product.
    # Here we take just the mean los azimuth value of all L2B wind results of the orbit within our bounding box.
    # If you are looking at whole orbits this would need to be modified.
    los_azimuth = int(np.mean(ds_L2B[L2B_channel + "_wind_result_los_azimuth"][~mask].load()))
    z_am = -ds_aux_met["layer_wind_component_u_off_nadir"] * np.sin(
        np.deg2rad(los_azimuth)
    ) - ds_aux_met["layer_wind_component_v_off_nadir"] * np.cos(np.deg2rad(los_azimuth))
    z_am = z_am[:, ymax - 1:]

    # -------------- create L2B patches ----------------------------------
    patches = []
    for x0, x1, y0, y1 in zip(X0, X1, Y0, Y1):
        patches.append(((x0, y0), (x0, y1), (x1, y1), (x1, y0)))

    # define min and max value for the colorbar
    if vmin is None:
        # if L2B_parameter == "wind_result_wind_velocity":
        if "wind_result_wind_velocity" in L2B_parameter:
            vmin = -np.amax(np.abs(np.asarray([np.nanpercentile(Z, 2), np.nanpercentile(Z, 98)])))
        else:
            vmin = np.nanpercentile(Z, 1)
    if vmax is None:
        # if parameter == "wind_result_wind_velocity":
        if "wind_result_wind_velocity" in L2B_parameter:
            vmax = np.amax(np.abs(np.asarray([np.nanpercentile(Z, 2), np.nanpercentile(Z, 98)])))
        else:
            vmax = np.nanpercentile(Z, 99)

    # ---------------------------- plot L2B and AUX_MET orbit ------------------
    fig = plt.figure(figsize=(8, 12), constrained_layout=True)
    gs = fig.add_gridspec(3, 1)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[1, 0], sharex=ax1, sharey=ax1)
    ax_map = fig.add_subplot(gs[2, 0], projection=ccrs.PlateCarree(central_longitude=10))

    coll = PolyCollection(
        patches,
        array=Z,
        cmap=cm.RdBu_r,
        norm=colors.Normalize(
            vmin=vmin,
            vmax=vmax,
            clip=False,
        ),
    )
    ax1.add_collection(coll)
    ax1.set_ylabel("Altitude [km]")
    ax1.set_title(
        "L2B {} - {} \n {} wind results \n orbit_no = {}".format(
            L2B_channel.title(), L2B_parameter, len(Z), orbit_no
        )
    )
    ax1.grid()

    locator = mdates.AutoDateLocator(minticks=4, maxticks=8)
    formatter = mdates.ConciseDateFormatter(locator)
    ax1.xaxis.set_major_locator(locator)
    ax1.xaxis.set_major_formatter(formatter)
    ax1.autoscale()
    ax1.set_ylim(-1, max_altitude)
    ax1.tick_params(labelbottom=False)

    ax2.pcolormesh(x_am, y_am.T, z_am[:-1, :].T, vmin=vmin, vmax=vmax, cmap="RdBu_r")
    ax2.set_ylim(-1, max_altitude)
    ax2.set_xlabel("Date [UTC]")
    ax2.set_ylabel("Altitude [km]")
    ax2.set_title("AUX_MET {}".format("HLOS wind"))
    ax2.grid()
    locator = mdates.AutoDateLocator(minticks=4, maxticks=8)
    formatter = mdates.ConciseDateFormatter(locator)
    ax2.xaxis.set_major_locator(locator)
    ax2.xaxis.set_major_formatter(formatter)

    ax_map.stock_img()
    ax_map.gridlines(draw_labels=True, linewidth=0.3, color="black", alpha=0.5, linestyle="-")
    ax_map.set_extent([-10, 30, 40, 65], crs=ccrs.PlateCarree())
    ax_map.scatter(
        ds_aux_met["longitude_off_nadir"],
        ds_aux_met["latitude_off_nadir"],
        marker="o",
        c="g",
        s=5,
        transform=ccrs.Geodetic(),
        label="AUX_MET",
    )
    ax_map.scatter(
        ds_L2B[L2B_channel + "_wind_result_start_longitude"],
        ds_L2B[L2B_channel + "_wind_result_start_latitude"],
        marker="o",
        c="r",
        s=5,
        transform=ccrs.Geodetic(),
        label="L2B {}".format(L2B_channel),
    )
    ax_map.legend()

    fig.colorbar(coll, ax=[ax1, ax2], aspect=50, pad=0.01, label="HLOS wind [cm/s]")

Now we plot an example orbit (here number 10)

In [ ]:
plot_wind_from_collocated_orbits(
    L2B_parameter='rayleigh_wind_result_wind_velocity',
    L2B_channel="rayleigh",
    L2B_obs_type="clear",
    L2B_error_estimate_threshold=850,
    L2B_QC_filter=True,
    orbit_no=10,
    vmin=-3000,
    vmax=3000,
)

It is obvious that the spatial resolution of the AUX_MET model data is much higher in the vertical as well as in the horizontal domain.
The easiest way to find collocated AUX_MET measurements for each L2B wind group would be a nearest neighbour approach. This, however, would not account for possible wind gradients which are covered by the highly resolved model data but not by the coarser L2B measurement grid. 
Thus, we average all AUX_MET wind data (here we use center locations) within one L2B measurement group.
This is done for the AUX_MET u and v wind component which are then used to calculate the horizontal line of sight winds by using the line of sight azimuth angle.  
$HLOS=-u*\sin(\phi)-v*\cos(\phi)$  
with $\phi$ as L2B azimuth angle.

In [ ]:
def spatial_collocate_L2B_AUX_MET_fast(
    L2B_aux_met_u,
    L2B_aux_met_v,
    L2B_lat_start,
    L2B_lat_stop,
    L2B_altitude_bottom,
    L2B_altitude_top,
    L2B_los_azimuth,
    aux_met_lat,
    aux_met_altitude,
    aux_met_u,
    aux_met_v,
):

    # create mask where aux_met data is in between one L2B observation
    # We have to distinguish between ascending and descending orbits
    if (L2B_lat_stop - L2B_lat_start).mean() > 0:
        AM_within_L2B_mask = (
            (
                (L2B_lat_stop[:, None] > aux_met_lat[None, :])
                & (L2B_lat_start[:, None] < aux_met_lat[None, :])
            )[:, :, None]
            & (L2B_altitude_bottom[:, None, None] < aux_met_altitude[None, :, :])
            & (L2B_altitude_top[:, None, None] > aux_met_altitude[None, :, :])
        )
    else:
        AM_within_L2B_mask = (
            (
                (L2B_lat_stop[:, None] < aux_met_lat[None, :])
                & (L2B_lat_start[:, None] > aux_met_lat[None, :])
            )[:, :, None]
            & (L2B_altitude_bottom[:, None, None] < aux_met_altitude[None, :, :])
            & (L2B_altitude_top[:, None, None] > aux_met_altitude[None, :, :])
        )
    only_one_meas_per_group_mask = np.all(~AM_within_L2B_mask, axis=(1, 2))

    # average where latitude difference is large enough and thus aux_met data could be found within one L2B group
    L2B_aux_met_u[~only_one_meas_per_group_mask] = np.average(
        np.broadcast_to(
            aux_met_u,
            (AM_within_L2B_mask[~only_one_meas_per_group_mask].shape),
        ),
        axis=(1, 2),
        weights=AM_within_L2B_mask[~only_one_meas_per_group_mask],
    )
    L2B_aux_met_v[~only_one_meas_per_group_mask] = np.average(
        np.broadcast_to(
            aux_met_v,
            (AM_within_L2B_mask[~only_one_meas_per_group_mask].shape),
        ),
        axis=(1, 2),
        weights=AM_within_L2B_mask[~only_one_meas_per_group_mask],
    )

    # We have to account for cases where one observation consists only of one measurement,
    # which results in an observation which has the same start and stop latitude and thus don't allow us to find collocated AUX_MET data with the procedure above.
    # These cases have to be treated extra. We just apply a nearest neighbour approach for them.
    # create mask with size where we have only one measurement per group and the number of AUX_MET profiles and set all to False
    AM_nearest_L2B_geolocated_mask = np.zeros(
        (only_one_meas_per_group_mask.sum(), len(aux_met_lat)),
        dtype=bool,
    )

    # Set the nearest AUX_MET profiles to True
    AM_nearest_L2B_geolocated_mask[
        np.arange(AM_nearest_L2B_geolocated_mask.shape[0]),
        np.argmin(
            np.abs(L2B_lat_start[only_one_meas_per_group_mask][:, None] - aux_met_lat),
            axis=1,
        ),
    ] = True

    # Combine the mask with the vertical collocation
    AM_nearest_L2B_mask = (
        AM_nearest_L2B_geolocated_mask[:, :, None]
        & (
            L2B_altitude_bottom[only_one_meas_per_group_mask][:, None, None]
            < aux_met_altitude[None, :, :]
        )
        & (
            L2B_altitude_top[only_one_meas_per_group_mask][:, None, None]
            > aux_met_altitude[None, :, :]
        )
    )

    # determine indices where we now have found AUX MET data for L2B groups.
    # However, we still have L2B measurements, where no data could be colloceted. These muss be accounted for when we now assign the collocated AUX_MET data to the L2B data variable.
    # The L2B measurements where we couldn't find collocated AUX_MET data will now be set to nan.
    idx_valid = np.all(~AM_nearest_L2B_mask, axis=(1, 2))
    idx_aux_met_nearest_L2B = np.flatnonzero(only_one_meas_per_group_mask)[~idx_valid]

    L2B_aux_met_u[idx_aux_met_nearest_L2B] = np.average(
        np.broadcast_to(
            aux_met_u,
            (AM_nearest_L2B_mask[~idx_valid].shape),
        ),
        axis=(1, 2),
        weights=AM_nearest_L2B_mask[~idx_valid],
    )
    L2B_aux_met_v[idx_aux_met_nearest_L2B] = np.average(
        np.broadcast_to(
            aux_met_v,
            (AM_nearest_L2B_mask[~idx_valid].shape),
        ),
        axis=(1, 2),
        weights=AM_nearest_L2B_mask[~idx_valid],
    )

    L2B_aux_met_hlos = -L2B_aux_met_u * np.sin(
        np.deg2rad(L2B_los_azimuth)
    ) - L2B_aux_met_v * np.cos(np.deg2rad(L2B_los_azimuth))

    return L2B_aux_met_hlos

Now we have a fast collocation procedure without any loops.
This has now to be applied to all collocated orbits.
Since indexing operations on dask and xarray-datasets are very slow, we extract the necessary parameter arrays.

In [ ]:
def apply_spatial_collocate_L2B_AUX_MET(ds_dict, channel):

    # First load the dask arrays to obtain the xarray datasets.
    # Here we just use the necessary parameters to keep the memory usage low.
    ds_aux_met_loaded = ds_dict["aux_met"][
        [
            "latitude_off_nadir",
            "layer_altitude_off_nadir",
            "layer_wind_component_u_off_nadir",
            "layer_wind_component_v_off_nadir",
            "orbit_mask",
        ]
    ].load()
    ds_L2B_loaded = ds_dict[channel][
        [
            channel + "_wind_result_start_latitude",
            channel + "_wind_result_stop_latitude",
            channel + "_wind_result_bottom_altitude",
            channel + "_wind_result_top_altitude",
            channel + "_wind_result_los_azimuth",
            "orbit_mask",
            "orbit_aux_met_idx",
        ]
    ].load()

    # Create a list to which we append all collocated HLOS values
    L2B_aux_met_hlos = []
    L2B_aux_met_idx = ds_L2B_loaded["orbit_aux_met_idx"].values

    # In the following we loop over all orbit.
    for orbit_no in tqdm_notebook(range(ds_L2B_loaded.dims["orbit_no"])):
        # Instead of a whole xarray dataset, only the parameter values are used.
        L2B_orbit_idx = (ds_L2B_loaded["orbit_mask"] == orbit_no).values
        L2B_lat_start = ds_L2B_loaded[channel + "_wind_result_start_latitude"][L2B_orbit_idx].values
        L2B_lat_stop = ds_L2B_loaded[channel + "_wind_result_stop_latitude"][L2B_orbit_idx].values
        L2B_altitude_bottom = ds_L2B_loaded[channel + "_wind_result_bottom_altitude"][
            L2B_orbit_idx
        ].values
        L2B_altitude_top = ds_L2B_loaded[channel + "_wind_result_top_altitude"][
            L2B_orbit_idx
        ].values
        L2B_los_azimuth = ds_L2B_loaded[channel + "_wind_result_los_azimuth"][L2B_orbit_idx].values

        L2B_aux_met_u = np.full(L2B_lat_start.shape, np.nan)
        L2B_aux_met_v = np.full(L2B_lat_start.shape, np.nan)

        aux_met_orbit_no = L2B_aux_met_idx[orbit_no]
        aux_met_orbit_idx = (ds_aux_met_loaded["orbit_mask"] == aux_met_orbit_no).values
        aux_met_lat = ds_aux_met_loaded["latitude_off_nadir"][aux_met_orbit_idx].values
        aux_met_altitude = (
            ds_aux_met_loaded["layer_altitude_off_nadir"][aux_met_orbit_idx].values / 100.0
        )
        aux_met_u = ds_aux_met_loaded["layer_wind_component_u_off_nadir"][aux_met_orbit_idx].values
        aux_met_v = ds_aux_met_loaded["layer_wind_component_v_off_nadir"][aux_met_orbit_idx].values

        # append the collocated HLOS values which are returned by the spatial_collocate_L2B_AUX_MET_fast function to the list
        L2B_aux_met_hlos.extend(
            spatial_collocate_L2B_AUX_MET_fast(
                L2B_aux_met_u,
                L2B_aux_met_v,
                L2B_lat_start,
                L2B_lat_stop,
                L2B_altitude_bottom,
                L2B_altitude_top,
                L2B_los_azimuth,
                aux_met_lat,
                aux_met_altitude,
                aux_met_u,
                aux_met_v,
            )
        )
    # Add the collocated and hlos-converted AUX_MET values to the dataset
    ds_dict[channel]["L2B_aux_met_hlos"] = (
        (list(ds_L2B_loaded.dims)[0]),
        np.asarray(L2B_aux_met_hlos),
    )
    return ds_dict

Add the collocated and hlos-converted AUX_MET values to the datasets by applying the collocation function.  
This takes around 3 minutes for all Rayleigh wind measurements for the entire mission and around 16 minutes for all Mie measurements. 

In [ ]:
ds_dict = apply_spatial_collocate_L2B_AUX_MET(ds_dict, 'rayleigh')

In [ ]:
ds_dict = apply_spatial_collocate_L2B_AUX_MET(ds_dict, 'mie')

Now we can test the spatial collocation of AUX_MET and L2B wind groups by using the plotting function from above and passing the 'L2B_aux_met_hlos' wind parameter

In [ ]:
plot_wind_from_collocated_orbits(
    L2B_parameter='L2B_aux_met_hlos',
    L2B_channel="rayleigh",
    L2B_obs_type="clear",
    L2B_error_estimate_threshold=850,
    L2B_QC_filter=True,
    orbit_no=10,
    vmin=-3000,
    vmax=3000,
)

#### Save datasets with collocated data to user space
This allows analyzing the data further without repeating the collocation process if the notebook kernel is started again.
This will save a lot of time.  
Just uncomment the cell where the datasets are saved with the 'to_netcdf'-method.

In [ ]:
# Define file paths
user_folder = os.path.expanduser("~")
results_folder = os.path.join(user_folder, "files/results")
os.makedirs(results_folder, exist_ok=True)
ds_dict_rayleigh_folder = os.path.join(results_folder, 'rayleigh_aux_met_collocated.nc')
ds_dict_mie_folder = os.path.join(results_folder, 'mie_aux_met_collocated.nc')

In [ ]:
# # save to netCDF
# ds_dict['rayleigh'].to_netcdf(ds_dict_rayleigh_folder)
# ds_dict['mie'].to_netcdf(ds_dict_mie_folder)

If datasets have already been saved to the user space you can use the following to Load them again (uncomment).

In [ ]:
# ds_dict = {
#     "rayleigh": xr.open_dataset(ds_dict_rayleigh_folder),
#     "mie": xr.open_dataset(ds_dict_mie_folder),
# }

## Filter, analyze and plot data
### Interactive scatter plot of L2B and AUX_MET comparison

Add timestamps for further processing to the datasets.  
Create a new variable based on L2B start times by adding the time difference between 01.01.1970 and 01.01.2001 as seconds.

In [ ]:
# add datetime timestamp
datetime_difference = (datetime(2000, 1, 1) - datetime(1970, 1, 1)).total_seconds()
ds_dict["rayleigh"]["rayleigh_wind_result_start_time_timestamp"] = (
    (list(ds_dict["rayleigh"].dims)[0]),
    ds_dict["rayleigh"]["rayleigh_wind_result_start_time"].data + datetime_difference,
)
ds_dict["mie"]["mie_wind_result_start_time_timestamp"] = (
    (list(ds_dict["mie"].dims)[0]),
    ds_dict["mie"]["mie_wind_result_start_time"].data + datetime_difference,
)

Now we define a class for filtering, analyzing and plotting the data.  
The filtering can also be done with an interactive scatter plot.  
Based on the filtered data we can create a time series of the systematic and random error of L2B wind measurements.

In [ ]:
class Analyze_and_Plot:
    """
    Class for plotting and analyzing colocated data with scatter plot and ipywidgets-interact
    """

    def __init__(self, ds_dict, channel):
        self.ds_L2B = ds_dict[channel].load()
        self.channel = channel
        self.mask = None
        self.wind_difference = None
        self.orbit_type = None

    def filter_data(
        self,
        obs_type,
        validity_flag,
        orbit_type,
        filter_hot_pixel,
        filter_blacklisted,
        err_estimate_range,
        time_range,
        altitude_range,
        which_range_bin_range,
    ):
        """
        Method to filter L2B measurements for different parameters.

        Parameters
        ----------
        obs_type : 'str'
            obs_type can be clear or cloudy
        validity_flag : 'int'
            validity flag can be 0 (invalid) or 1 (valid)
        orbit_type : 'str'
            orbit_type can be ascending or descending
        filter_hot_pixel : 'int'
            filter_hot_pixels can be 0 to not filter or 1 to filter for hot pixels
        filter_blacklisted : 'int'
            filter_blacklisted can be 0 to not filter or 1 to filter for blacklisted data
        err_estimate_range : 'tuple' of int or float
            err_estimate_range is a tuple consisting of two elements with (err_estimate_min, err_estimate_max) in cm/s
        time_range : 'tuple' of int or float
            time_range is a tuple consisting of two elements with (time_range_min, time_range_max) as timestamps
        altitude_range : 'tuple' of int or float
            altitude_range is a tuple consisting of two elements with (altitude_range_min, alttude_range_max) in meter
        which_range_bin_range : 'tuple' of int or float
            which_range_bin_range is a tuple consisting of two elements with (which_range_bin_range_min, which_range_bin_range_max)

        Returns
        -------
        mask : numpy boolean array
            mask with same length as self.ds_L2B

        """

        if orbit_type == "ascending":
            azimuth_range = (180, 360)
        elif orbit_type == "descending":
            azimuth_range = (0, 180)
        elif orbit_type == "both":
            azimuth_range = (0, 360)

        range_dict = {
            "wind_result_bottom_altitude": altitude_range,
            "wind_result_HLOS_error": err_estimate_range,
            "wind_result_range_bin_number": which_range_bin_range,
            "wind_result_los_azimuth": azimuth_range,
            "wind_result_start_time_timestamp": time_range,
        }

        # create mask based on filters
        # mask all nan values
        mask = np.isnan(self.ds_L2B["L2B_aux_met_hlos"])

        # mask invalid data
        mask = mask | (self.ds_L2B[self.channel + "_wind_result_validity_flag"] != validity_flag)

        # mask cloudy or clear values
        if obs_type == "cloudy":
            mask = mask | (self.ds_L2B[self.channel + "_wind_result_observation_type"] != 1)
        elif obs_type == "clear":
            mask = mask | (self.ds_L2B[self.channel + "_wind_result_observation_type"] != 2)

        for key, value in range_dict.items():
            mask = (
                mask
                | (self.ds_L2B[self.channel + "_" + key] < value[0])
                | (self.ds_L2B[self.channel + "_" + key] > value[1])
            )

        # filter Hot pixel
        hot_pixels_rayleigh = {
            "1": (
                11,
                datetime.strptime("2018-09-07", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
            "2": (
                5,
                datetime.strptime("2018-11-04", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
            "3": (
                15,
                datetime.strptime("2018-11-24", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
            "4": (
                20,
                datetime.strptime("2019-01-27", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
            "5": (
                1,
                datetime.strptime("2019-02-20", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
            "6": (
                11,
                datetime.strptime("2019-03-17", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
            "7": (
                3,
                datetime.strptime("2019-05-08", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
        }

        hot_pixels_mie = {
            "1": (
                16,
                datetime.strptime("2018-08-22", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
            "2": (
                24,
                datetime.strptime("2018-08-22", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
            "3": (
                13,
                datetime.strptime("2018-10-21", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
            "4": (
                2,
                datetime.strptime("2018-10-24", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
            "5": (
                5,
                datetime.strptime("2019-01-09", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
            "6": (
                20,
                datetime.strptime("2019-03-31", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
            "7": (
                10,
                datetime.strptime("2019-04-26", "%Y-%m-%d")
                .replace(tzinfo=timezone.utc)
                .timestamp(),
            ),
        }

        hot_pixel_correct_date = (
            datetime.strptime("2019-06-14", "%Y-%m-%d").replace(tzinfo=timezone.utc).timestamp()
        )

        if self.channel == "rayleigh":
            hot_pixels = hot_pixels_rayleigh
        elif self.channel == "mie":
            hot_pixels = hot_pixels_mie

        if bool(filter_hot_pixel) is True:
            for key, value in hot_pixels.items():
                mask = mask | (
                    (self.ds_L2B[self.channel + "_wind_result_range_bin_number"] == value[0])
                    & (self.ds_L2B[self.channel + "_wind_result_start_time_timestamp"] > value[1])
                    & (
                        self.ds_L2B[self.channel + "_wind_result_start_time_timestamp"]
                        < hot_pixel_correct_date
                    )
                )

        # end filter hot pixel ###

        # filter exlusion list
        # Especially for the newest baselines including the reprocessed datasets, blacklistet periods are already flagged as invalid.
        # However, for older baselines with laser FM-A, the following filtering helps to exclude measurements which can be of bad quality.

        if bool(filter_blacklisted) is True:

            exclusion_dates_list = [
                ["2018-09-25", "03:00:00", "2018-09-26", "15:00:00"],
                ["2018-12-13", "02:00:00", "2018-12-13", "16:50:00"],
                ["2019-01-21", "00:00:00", "2019-01-21", "23:59:59"],
                ["2019-02-18", "14:00:00", "2019-02-20", "09:20:00"],
                ["2019-02-26", "12:50:00", "2019-02-27", "16:37:00"],
                ["2019-06-16", "19:00:00", "2019-06-28", "13:54:23"],
                ["2019-07-01", "18:17:00", "2019-07-02", "12:15:23"],
                ["2019-07-08", "04:00:00", "2019-07-08", "23:45:00"],
                ["2019-07-15", "04:00:00", "2019-07-16", "01:00:00"],
                ["2019-07-23", "00:30:00", "2019-07-23", "05:47:21"],
                ["2020-01-15", "11:08:59", "2020-01-15", "11:30:38"],
                ["2020-03-04", "00:32:00", "2020-03-04", "05:47:21"],
                ["2020-03-06", "06:45:00", "2020-03-06", "09:32:35"],
                ["2020-03-09", "21:00:00", "2020-03-12", "03:59:47"],
                ["2020-05-25", "07:54:00", "2020-05-27", "20:26:30"],
                ["2020-07-06", "04:00:00", "2020-07-10", "23:59:59"],
                ["2020-09-21", "00:04:00", "2020-09-24", "03:05:00"],
                ["2020-10-05", "01:59:00", "2020-10-07", "04:00:00"],
                ["2020-10-08", "23:59:00", "2020-10-10", "01:00:00"],
                ["2020-11-17", "16:00:00", "2020-11-20", "15:00:00"],
                ["2020-11-24", "16:00:00", "2020-11-26", "23:59:59"],
                ["2020-12-04", "08:00:00", "2020-12-08", "10:00:00"],
                ["2020-12-04", "08:00:00", "2020-12-14", "08:30:00"],
                ["2021-01-14", "01:13:00", "2021-01-14", "07:26:00"],
                ["2021-01-22", "10:02:00", "2021-01-27", "23:59:59"],
                ["2021-01-31", "00:25:00", "2021-02-05", "12:30:00"],
                ["2021-02-12", "10:05:00", "2021-02-15", "12:30:00"],
                ["2021-02-26", "06:00:00", "2021-03-01", "12:30:00"],
                ["2021-03-09", "08:45:00", "2021-03-11", "23:59:59"],
                ["2021-03-16", "08:45:00", "2021-03-18", "23:59:59"],
                ["2021-03-29", "00:00:00", "2021-05-06", "12:00:00"],
                ["2021-05-11", "12:58:00", "2021-05-11", "18:20:00"],
                ["2021-05-17", "15:00:00", "2021-05-22", "23:59:59"],
                ["2021-05-24", "19:00:00", "2021-05-28", "23:59:59"],
                ["2021-06-21", "14:50:00", "2021-06-26", "12:00:00"],
                ["2021-10-05", "00:00:00", "2021-10-05", "23:59:59"],
                ["2021-10-18", "11:20:00", "2021-11-02", "11:20:00"],
                ["2021-11-22", "15:00:00", "2021-11-27", "12:00:00"],
                ["2021-11-29", "15:00:00", "2021-12-04", "11:00:00"],
            ]

            exclusion_dates = {}
            for i in range(len(exclusion_dates_list)):
                exclusion_dates[str(i)] = (
                    datetime.strptime(" ".join(exclusion_dates_list[i][0:2]), "%Y-%m-%d %H:%M:%S")
                    .replace(tzinfo=timezone.utc)
                    .timestamp(),
                    datetime.strptime(" ".join(exclusion_dates_list[i][2:4]), "%Y-%m-%d %H:%M:%S")
                    .replace(tzinfo=timezone.utc)
                    .timestamp(),
                )

            for key, value in exclusion_dates.items():
                mask = mask | (
                    (self.ds_L2B[self.channel + "_wind_result_start_time_timestamp"] > value[0])
                    & (self.ds_L2B[self.channel + "_wind_result_start_time_timestamp"] < value[1])
                )

        # end filter exclusion list

        return mask

    def return_masked_hlos_winds(self, mask):
        """
        Return masked L2B and AUX_MET HLOS winds in m/s
        """
        L2B_aux_met_hlos = self.ds_L2B["L2B_aux_met_hlos"][~mask].values / 100.0
        L2B_hlos = self.ds_L2B[self.channel + "_wind_result_wind_velocity"][~mask].values / 100.0

        return L2B_aux_met_hlos, L2B_hlos

    def analyze_and_plot(
        self,
        obs_type,
        validity_flag,
        orbit_type,
        filter_hot_pixel,
        filter_blacklisted,
        err_estimate_range,
        time_range,
        altitude_range,
        which_range_bin_range,
    ):
        """
        Interactive plotting of collocated L2B and AUX_MET measurements

        Parameters
        ----------
        See filter_data method

        Returns
        -------
        Matplotlib plot
        """

        self.obs_type = obs_type
        self.orbit_type = orbit_type
        self.mask = self.filter_data(
            obs_type,
            validity_flag,
            orbit_type,
            filter_hot_pixel,
            filter_blacklisted,
            err_estimate_range,
            time_range,
            altitude_range,
            which_range_bin_range,
        )
        L2B_aux_met_hlos, L2B_hlos = self.return_masked_hlos_winds(self.mask)
        self.wind_difference = L2B_hlos - L2B_aux_met_hlos

        # calculate statistics (systematic and random errors, regression line)
        slope, intercept, r_value, p_value, stderr = linregress(L2B_aux_met_hlos, L2B_hlos)
        linreg_y = intercept + slope * L2B_aux_met_hlos
        wind_difference_mean = self.wind_difference.mean()
        wind_difference_std = self.wind_difference.std()
        wind_difference_median = np.median(self.wind_difference)
        MAD = 1.4826 * np.median(np.abs(self.wind_difference - wind_difference_median))

        # create string to show as text box in scatterplot
        textstr = "\n".join(
            (
                "bias = {:.2f} m/s".format(wind_difference_mean),
                "stddev = {:.2f} m/s".format(wind_difference_std),
                "slope = {:.2f}".format(slope),
                "intercept = {:.2f} m/s".format(intercept),
                "1.4826*MAD = {:.2f} m/s".format(MAD),
                "r = {:.3f}".format(r_value),
            )
        )

        # clear axis and plot data
        self.ax.clear()
        self.ax.scatter(L2B_aux_met_hlos, L2B_hlos, s=5, alpha=0.2, linewidths=0, zorder=2)
        self.ax.plot([-100, 100], [-100, 100], "k", label="1:1", zorder=1)
        self.ax.plot(L2B_aux_met_hlos, linreg_y, "r", label="regression line", zorder=3)
        self.ax.grid()
        self.ax.set_xlabel("AUX_MET wind speed HLOS [m/s]")
        self.ax.set_ylabel("Aeolus wind speed HLOS [m/s]")
        self.ax.set_xlim(-100, 100)
        self.ax.set_ylim(-100, 100)

        # place text in scatterplot
        self.ax.text(
            0.03,
            0.97,
            textstr,
            transform=self.ax.transAxes,
            verticalalignment="top",
            bbox=dict(facecolor="white", alpha=0.9, boxstyle="round"),
        )

        self.ax_hist.clear()
        self.ax_hist.hist(self.wind_difference, bins=np.arange(-25, 25, 0.5), density=True)
        self.ax_hist.grid()
        self.ax_hist.set_ylim(0, 0.1)
        self.ax_hist.set_xlabel("L2B - AUX_MET wind speed difference [m/s]")
        # self.ax.set_ylabel("Aeolus wind speed HLOS [m/s]")
        self.fig.suptitle(
            "Aeolus-AUX_MET comparison \n {} measurements \n {} to {}".format(
                len(L2B_aux_met_hlos),
                datetime.utcfromtimestamp(time_range[0]).strftime("%Y-%m-%d"),
                datetime.utcfromtimestamp(time_range[1]).strftime("%Y-%m-%d"),
            )
        )

        self.fig.tight_layout()

    def create_figure(self):
        """Create matplitlib figure"""
        self.fig, (self.ax, self.ax_hist) = plt.subplots(1, 2, figsize=(10, 5))

    def plot_interactive(self):
        """Interactive filtering and plotting of data"""
        # Define the time range for the available data
        time_range_available_str = np.arange(
            np.array(
                self.ds_L2B[self.channel + "_wind_result_start_time_timestamp"][0],
                dtype="datetime64[s]",
            ),
            np.array(
                self.ds_L2B[self.channel + "_wind_result_start_time_timestamp"][-1],
                dtype="datetime64[s]",
            )
            + np.timedelta64(2, "D"),
            dtype="datetime64[D]",
        )
        time_range_available_ts = (
            time_range_available_str - np.datetime64("1970-01-01T00:00:00")
        ) / np.timedelta64(1, "s")
        time_range_available = list(zip(time_range_available_str, time_range_available_ts))

        # Create the matplotlib figure
        self.create_figure()
        self.analyze_and_plot(
            obs_type="clear",
            validity_flag=1,
            orbit_type="both",
            filter_hot_pixel=1,
            filter_blacklisted=1,
            err_estimate_range=(0, 800),
            time_range=(time_range_available[0][1], time_range_available[-1][1]),
            altitude_range=(0, 30000),
            which_range_bin_range=(0, 24),
        )

        # Use the ipywidgets interact functionality
        interact(
            self.analyze_and_plot,
            obs_type=widgets.Dropdown(
                options=["cloudy", "clear"], value="clear", style={"description_width": "initial"}
            ),
            validity_flag=widgets.RadioButtons(
                options=[("valid", 1), ("invalid", 0)],
                value=1,
                style={"description_width": "initial"},
            ),
            orbit_type=widgets.RadioButtons(
                options=["ascending", "descending", "both"],
                value="both",
                style={"description_width": "initial"},
            ),
            filter_hot_pixel=widgets.RadioButtons(
                options=[("yes", 1), ("no", 0)],
                value=1,
                style={"description_width": "initial"},
            ),
            filter_blacklisted=widgets.RadioButtons(
                options=[("yes", 1), ("no", 0)],
                value=1,
                style={"description_width": "initial"},
            ),
            err_estimate_range=widgets.IntRangeSlider(
                value=(0, 800),
                min=0,
                max=5000,
                step=1,
                continuous_update=False,
                style={"description_width": "initial"},
                layout={"width": "400px"},
            ),
            time_range=widgets.SelectionRangeSlider(
                options=time_range_available,
                # index = (0, -1),
                value=(time_range_available[0][1], time_range_available[-1][1]),
                continuous_update=False,
                style={"description_width": "initial"},
                layout={"width": "400px"},
            ),
            altitude_range=widgets.IntRangeSlider(
                value=(1000, 30000),
                min=-500,
                max=30000,
                continuous_update=False,
                style={"description_width": "initial"},
                layout={"width": "400px"},
            ),
            which_range_bin_range=widgets.IntRangeSlider(
                value=(0, 24),
                min=0,
                max=24,
                continuous_update=False,
                style={"description_width": "initial"},
                layout={"width": "400px"},
            ),
        )

    def create_pandas_dataframe(self, data, mask):
        """
        Create pandas dataframe with L2B start time as index
        """
        pd_df = pd.DataFrame(
            data,
            index=pd.to_datetime(
                self.ds_L2B[self.channel + "_wind_result_start_time_timestamp"][~mask].values,
                unit="s",
            ),
        ).sort_index()

        return pd_df

    def create_time_series(self, statistic_window_length=1, averaging_window_length=7):
        """Create time series based on the interactively filtered data"""
        swl = statistic_window_length
        awl = averaging_window_length
        min_number_days = round(awl / swl / 2)

        pd_df = self.create_pandas_dataframe(self.wind_difference, self.mask)

        fig, ax = plt.subplots(
            3,
            1,
            figsize=(8, 6),
            sharex=True,
            constrained_layout=True,
            gridspec_kw={"height_ratios": [3, 3, 2]},
        )

        # If resampled to one day, we can set the rolling window to 7 if we want to apply it on 7 days for example.
        # This allows us to set center=True which is not possible if setting the rolling window to "7D" (7 days) for example.
        # In this case, the results would be shifted because the rolling window computations would be not centered.
        # So be careful when doing these calculations.
        pd_df.resample(f"{swl}D").mean().rolling(
            awl, min_periods=min_number_days, center=True
        ).median().plot(kind="line", ax=ax[0], ylim=(-6, 6))
        pd_df.resample(f"{swl}D").std().rolling(
            awl, min_periods=min_number_days, center=True
        ).median().plot(kind="line", ax=ax[1], ylim=(0, 9))
        (pd_df.resample(f"{swl}D").apply(median_abs_deviation) * 1.4826).rolling(
            awl, min_periods=min_number_days, center=True
        ).median().plot(kind="line", ax=ax[1], ylim=(0, 9))
        pd_df.resample(f"{swl}D").count().rolling(
            awl, min_periods=min_number_days, center=True
        ).median().plot(kind="line", ax=ax[2], legend=False)

        ax[0].axhline(0, c="k", lw=1.0, ls="--")
        ax[2].set_ylim(bottom=0)
        ax[0].legend([f"{self.orbit_type} orbits"])
        ax[1].legend(["stddev", "1.4826*MAD"])
        ax[0].grid(which="both")
        ax[1].grid(which="both")
        ax[2].grid(which="both")
        ax[2].set_xlabel("Date")
        ax[0].set_ylabel("Bias [m/s]")
        ax[1].set_ylabel("Random error [m/s]")
        ax[2].set_ylabel(f"Counts per {swl} days")
        fig.suptitle(
            f"{self.channel.title()} - {self.obs_type} measurements \n Number of days for moving statistics = {swl}"
        )
        fig.align_ylabels()

    def create_time_series_with_orbit_types(
        self, statistic_window_length=1, averaging_window_length=7
    ):
        """Function which creates a time series of the systematic and random error for ascending and descending orbits"""
        swl = statistic_window_length
        awl = averaging_window_length
        min_number_days = round(awl / swl / 2)

        if self.channel == "rayleigh":
            obs_type = "clear"
            err_estimate_range = (0, 800)
        elif self.channel == "mie":
            obs_type = "cloudy"
            err_estimate_range = (0, 500)

        mask_ascending = self.filter_data(
            obs_type=obs_type,
            validity_flag=1,
            orbit_type="ascending",
            filter_hot_pixel=1,
            filter_blacklisted=1,
            err_estimate_range=err_estimate_range,
            time_range=(datetime(2018, 9, 1).timestamp(), datetime(2022, 3, 1).timestamp()),
            altitude_range=(0, 30000),
            which_range_bin_range=(0, 24),
        )
        L2B_aux_met_hlos_asc, L2B_hlos_asc = self.return_masked_hlos_winds(mask_ascending)
        wind_difference_asc = L2B_hlos_asc - L2B_aux_met_hlos_asc
        pd_df_asc = self.create_pandas_dataframe(wind_difference_asc, mask_ascending)

        mask_descending = self.filter_data(
            obs_type=obs_type,
            validity_flag=1,
            orbit_type="descending",
            filter_hot_pixel=1,
            filter_blacklisted=1,
            err_estimate_range=err_estimate_range,
            time_range=(datetime(2018, 9, 1).timestamp(), datetime(2022, 3, 1).timestamp()),
            altitude_range=(0, 30000),
            which_range_bin_range=(0, 24),
        )
        L2B_aux_met_hlos_desc, L2B_hlos_desc = self.return_masked_hlos_winds(mask_descending)
        wind_difference_desc = L2B_hlos_desc - L2B_aux_met_hlos_desc
        pd_df_desc = self.create_pandas_dataframe(wind_difference_desc, mask_descending)

        fig, ax = plt.subplots(
            3,
            1,
            figsize=(8, 6),
            sharex=True,
            constrained_layout=True,
            gridspec_kw={"height_ratios": [3, 3, 2]},
        )

        pd_df_asc.resample(f"{swl}D").mean().rolling(
            awl, min_periods=min_number_days, center=True
        ).median().plot(kind="line", ax=ax[0], ylim=(-6, 6))
        (pd_df_asc.resample(f"{swl}D").apply(median_abs_deviation) * 1.4826).rolling(
            awl, min_periods=min_number_days, center=True
        ).median().plot(kind="line", ax=ax[1], ylim=(0, 9), legend=False)
        pd_df_asc.resample(f"{swl}D").count().rolling(
            awl, min_periods=min_number_days, center=True
        ).median().plot(kind="line", ax=ax[2], legend=False)

        pd_df_desc.resample(f"{swl}D").mean().rolling(
            awl, min_periods=min_number_days, center=True
        ).median().plot(kind="line", ax=ax[0], ylim=(-6, 6))
        (pd_df_desc.resample(f"{swl}D").apply(median_abs_deviation) * 1.4826).rolling(
            awl, min_periods=min_number_days, center=True
        ).median().plot(kind="line", ax=ax[1], ylim=(0, 9), legend=False)
        pd_df_desc.resample(f"{swl}D").count().rolling(
            awl, min_periods=min_number_days, center=True
        ).median().plot(kind="line", ax=ax[2], legend=False)

        ax[0].axhline(0, c="k", lw=1.0, ls="--")
        ax[2].set_ylim(bottom=0)
        ax[0].legend(["ascending", "descending"])
        ax[0].grid(which="both")
        ax[1].grid(which="both")
        ax[2].grid(which="both")
        ax[2].set_xlabel("Date")
        ax[0].set_ylabel("Bias [m/s]")
        ax[1].set_ylabel("1.4826*MAD [m/s]")
        ax[2].set_ylabel(f"Counts per {swl} days")
        fig.suptitle(
            f"{self.channel.title()} - {obs_type} measurements \n Number of days for moving statistics = {swl}"
        )
        fig.align_ylabels()
        fig.savefig("time_series.png")

Create the time series for Rayleigh measurements

In [ ]:
L2B_validation_rayleigh = Analyze_and_Plot(ds_dict, "rayleigh")

Interactive filtering

In [ ]:
L2B_validation_rayleigh.plot_interactive()

Create time series based on the interactively filtered data

In [ ]:
L2B_validation_rayleigh.create_time_series(statistic_window_length=1, averaging_window_length=7)

Create time series for ascending and descending orbits (not based on the interactively filtered data).  
See and adapt filter thresholds in the 'create_time_series_with_orbit_types'-method above.

In [ ]:
L2B_validation_rayleigh.create_time_series_with_orbit_types(statistic_window_length=2, averaging_window_length=3)

Create time series for ascending and descending orbits for Mie-cloudy measurements

In [ ]:
L2B_validation_mie = Analyze_and_Plot(ds_dict, "mie")
L2B_validation_mie.create_time_series_with_orbit_types(statistic_window_length=5, averaging_window_length=2)

#### Time series for the entire mission
To show the capability of this notebook, we defined a time period from 01.09.2018 until 01.02.2022 at the top of the notebook.
Results are shown in the following plots for Rayleigh-clear and Mie-cloudy measurements.
Keep in mind, if a applying such a long time period, this will take some time to run (about 1 hour for the entire mission).

In [ ]:
Rayleigh_time_series_encoded = b'...'
widgets.Image(value=b64decode(Rayleigh_time_series_encoded))

In [ ]:
Mie_time_series_encoded = b'...'
widgets.Image(value=b64decode(Mie_time_series_encoded))